In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, LSTM
from tensorflow.keras import utils
import tensorflow as tf
from tensorflow import keras
import h5py   

In [13]:
classifies = ['Audi','hyundai','lexus','mazda','Mercedes','opel','toyota','volkswagen','Bu']
classifies2 = classifies[:8]    #Loại lớp bù

path = "Class/"

In [21]:
data = []
labels = []
imagePaths = []

#Kích cỡ quy định cho các tấm ảnh trong lớp phải quy về kích cỡ chung với nhau
WIDTH = 128
HEIGHT = 128

N_CHANNELS = 3      #Kênh màu, Phải có 3 kênh RGB

#1.Lưu đường dẫn của hình ảnh và chỉ số thư mục chứa nó
for index, name in enumerate(classifies):
    for file in os.listdir(path+ name):
        imagePaths.append( [path+name +'/'+file, index])

#2.>>> Quan trọng : Xáo trộn sắp xếp ngẫu nhiên lại các phân tử mảng imagePaths[]
import random
random.shuffle(imagePaths)
print(imagePaths[:10])

#3. Đọc từng ảnh, thay đổi kích cỡ ảnh và lưu trữ chỉ số ảnh
for imagePath in imagePaths:
    image = cv2.imread(imagePath[0])            #Đọc ảnh từ đường dẫn
    image = cv2.resize(image, (WIDTH,HEIGHT))  #thay đổi kích cỡ hỉnh ảnh
    data.append(image)

    #Thêm lớp nhãn từ đường dẫn hình ảnh và cập nhật danh sách
    label = int(imagePath[1])
    labels.append(label)

print('Số lượng ảnh: {0}'.format(len(data)))

[['Class/Bu/hinh (97).jpg', 8], ['Class/opel/images85_rotate_90_260.jpg', 5], ['Class/volkswagen/images45_rotate_180_218.jpg', 7], ['Class/mazda/53f7746s-960_rotate_90_72.jpg', 3], ['Class/mazda/698ff3es-100_rotate_90_98.jpg', 3], ['Class/opel/images294_rotate_90_159.jpg', 5], ['Class/opel/opel-logo-sunroof-sticker-oto-cikartma__1318817497862971_rotate_180_285.jpg', 5], ['Class/opel/images302.jpg', 5], ['Class/mazda/11847_emblema-mazda-s-dvukhtsvetnoy-s_rotate_180_20.jpg', 3], ['Class/volkswagen/vwe11_rotate_180_305.jpg', 7]]
Số lượng ảnh: 7969


In [22]:

data = np.array(data, dtype='float')/255
labels = np.array(labels)
print(labels)

[8 5 7 ... 3 5 5]


Chia sẽ và huấn luyện dư liệu

In [27]:
#Chia dữ liệu ra làm 2 phần trong đó: train(70%) và test(30%)
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.3, random_state=50)

In [28]:
#Trước quy trình nhãn lớp

trainY = utils.to_categorical(trainY, num_classes= len(classifies))
#trainX = np.squeeze(trainX, axis=-1)

#Nếu kích cỡ ảnh resize càng lớn thì số hình ảnh huấn luyện càng ít. Ví dụ: 2100/128(kích cỡ ảnh resize) = 16; 2100/32 = 65
#Số lượng resize càng nhỏ thì chạy càng lâu => Chưa chắc hiệu quả
#Học ít thì mấy lớp gần giống nhau thì khó phân biệt. Còn học nhiều thì bị rối
#Nếu 500 tấm hình trên mỗi đối tường thì Epoch có gía trị tầm 30-35
#Nếu 500 tấm hình trên mỗi đối tường thì Epoch có gía trị tầm 120-280

print(trainX.shape)
print(testX.shape)
print(trainY.shape)
print(testY.shape)

#Các biến quan trọng

EPOCHS = 30
INIT_LR = 1e-3
BS = 32
#----------------
class_names = classifies

(5578, 128, 128, 3)
(2391, 128, 128, 3)
(5578, 9)
(2391,)


1.Mô hình kiến trúc CNN && Chạy Thủ và đánh giá

In [29]:

model = Sequential()  #khởi tạo một mô hình tuần tự .Kiểu mô hình tuần tự cho phép thêm các lớp mạng theo thứ tự tuần tự.

model.add(Convolution2D(32, (2,2), activation='relu', input_shape=(HEIGHT, WIDTH, N_CHANNELS)))

# Lớp này thực hiện phép lấy mẫu tối đa (max pooling) trên đầu ra của lớp trước đó. Pooling giúp giảm kích thước của dữ liệu và cải thiện khả năng chống nhiễu của mô hình
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(32, (2,2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

#Lớp Dropout() Này ngẫu nhiên bỏ 1 tỷ lệ các nỏ-ron ra trong quá trình huấn luyện, giúp ngăn ngừ mô hình học thuộc lòng (overfitting)
model.add(Dropout(0.25))

#Flatten(): Lớp này chuyển đổi dữ liệu đầu ra từ dạng 3 chiều (chiều cao, rộng, số kênh) thành dạng 1 chiều (vector), chuẩn bị cho các lớp Dense tiếp theo.
model.add(Flatten())

#Lớp Dense(): có 128 nơ-ron và sử dụng hàm kích hoạt ReLU.
model.add(Dense(128, activation='relu'))

#Lớp Dropout(0.5) hủy bỏ 50% các nơ-ron đầu ra trong quá trình huấn luyện.
model.add(Dropout(0.5))

#Lớp Dense():có 8 nơ-ron và sử dụng hàm kích hoạt softmax để cho ra xác suất thuộc một trong 8 lớp (output classes).
model.add(Dense(len(classifies), activation='softmax'))

#biên dịch mô hình. Quá trình biên dịch xác định hàm mất mát (loss function), bộ tối ưu hóa (optimizer) và các số liệu đánh giá (metrics) được sử dụng trong quá trình huấn luyện mô hình.
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 127, 127, 32)   │           416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 62, 62, 32)     │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 30752)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     3,936,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,941,831 (15.04 MB)

 Trainable params: 3,941,831 (15.04 MB)

 Non-trainable params: 0 (0.00 B)

None


In [30]:
#Huấn luyện mô hình CNN
model.fit(trainX, trainY, batch_size=BS, epochs=EPOCHS, verbose=1)

Epoch 1/30


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 9), output.shape=(None, 7)

2.Mô hình kiến trúc MobileNet && Chạy Thủ và đánh giá

3.Mô hình kiến trúc Lenet && Chạy Thủ và đánh giá

4.Mô hình kiến trúc InceptionV3 && Chạy Thủ và đánh giá